# Evaluation - Feature Importance 

### Objectives:
- Uncover the factors that significantly impact sales
- Build a machine learning model with a 90% accuracy rating (less than 10% average error)

---
## <u>Functions Used</u>

In [81]:
def grab_file():
    app = QApplication([])
    file_path, _ = QFileDialog.getOpenFileName(None, 
                                               "Select a CSV or XLSX file", "", 
                                               "Files (*.xlsx *.csv)")
    
    if not file_path:  # If no file is selected
        print("No file selected.")
        return None
    
    if file_path.endswith('.xlsx'):
        print("XLSX file selected. Converting to CSV.")
        df = pd.read_excel(file_path)
        csv_path = file_path.rsplit('.', 1)[0] + '.csv'
        df.to_csv(csv_path, index=False)
        print(f"CSV file created: {csv_path}")
    elif file_path.endswith('.csv'):
        print("CSV file selected. Reading file.")
        csv_path = file_path
        df = pd.read_csv(csv_path)
    else:
        raise ValueError("Unsupported file type selected.")
    return df

In [82]:
def data_cleaning(df):
    
    # Define the columns needed for analysis to simplify the dataset.
    cols_needed = ['Date', 'Category', 'Style', 'Size', 'SKU', 'Qty', 'Amount', 'promotion-ids', 'B2B']
    
    # Remove duplicates based on 'Order ID' to ensure unique transactions and select only the necessary columns.
    data = df.drop_duplicates(subset='Order ID')[cols_needed]
    
    # Standardize column names by renaming them to maintain consistent naming conventions across the dataset.
    data.rename(columns={'Qty': 'Quantity', 'promotion-ids': 'Promotions'}, inplace=True)
    
    # Fill missing values in 'Promotions' column with "No Promotion" to indicate transactions without any promotions.
    data['Promotions'].fillna("No Promotion", inplace=True)
    
    # Remove transactions with missing 'Amount' values as these represent incomplete data points.
    data.dropna(subset=['Amount'], inplace=True)
    data = data[(data['Quantity'] > 0) & (data['Amount'] > 0)]
    
    # Convert 'Amount' from INR to USD for standardization, assuming a fixed exchange rate (1 INR = 0.012 USD).
    data['Amount'] = data['Amount'] * 0.012
    
    # Simplify 'Promotions' column
    # Extract relevant information from Promotions
    promotions = ["Free-Financing", "Free Shipping", "Duplicated", "Coupon"]
    data['Promotions'] = data['Promotions'].apply(lambda x: next((promo for promo in promotions if promo in str(x)), x))
    
    # Convert "B2B" boolean to integer
    data['B2B'] = data['B2B'].astype(int)
    
    # Adjust 'Size' values
    data['Size'] = data['Size'].replace({'Free': 'Any'})

    # Convert the 'Date' column from string to DateTime objects for easier manipulation and analysis.
    data['Date'] = pd.to_datetime(data['Date'])
    
    # Remove numbers from 'Style'
    data['Style'] = data['Style'].str.replace('\d+', '', regex=True)
    
    # Sort by 'Date'
    data_sorted = data.sort_values(by='Date', ascending=True)
    
    # Return the cleaned dataset.
    return data_sorted

In [83]:
def feature_engineering(df):
    
    # Extract colors from 'SKU'
    color_names = ['MUSTARD', 'BLUE', 'NAVY', 'WHITE', 'BROWN', 'GREEN', 'BIEGE', 
                   'RED', 'PURPLE', 'MAROON', 'PINK', 'CHIKU', 'BLACK', 'GOLD', 
                   'BEIGE', 'ORANGE', 'YELLOW']
    
    # Extract the Colors from the column SKU
    df['SKU_Color'] = df['SKU'].apply(lambda sku: next((color for color in color_names if color in sku), 'NOT LISTED'))

    # Extracting Date (Year, Month, and Days)
#     data['Year'] = data['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day

    # Dropping features that have extracted information
    df = df.drop(['Date', 'SKU'], axis=1).reset_index(drop=True)
        
    # Return the engineered dataset.
    return df

In [84]:
def split_dataset(df):
    X = df.drop('Amount', axis=1)
    y = df['Amount']

    return train_test_split(X, y, test_size=0.2, random_state=42)

In [85]:
def preprocess_features(X_train, X_test):
    # Columns for one-hot encoding and label encoding
    ohe_cols = X_train.select_dtypes(include='object').columns.to_list()
    scale_cols = ['Quantity', 'Month', 'Day']
    
    # Preprocessor for transformations
    preprocessor = ColumnTransformer(
        transformers=[
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False), ohe_cols),
            ('scale', MinMaxScaler(), scale_cols)
        ], remainder='passthrough')  # 'B2B' remains unchanged
    
    # Fit and transform training data
    X_train_transformed = preprocessor.fit_transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)

    X_train_transformed = pd.DataFrame(X_train_transformed, columns = preprocessor.get_feature_names_out())
    X_test_transformed = pd.DataFrame(X_test_transformed, columns = preprocessor.get_feature_names_out())
    
    return X_train_transformed, X_test_transformed, preprocessor

In [86]:
def apply_transformation(target):
    """Transform the target variable to reduce skewness."""
    transformations = {
        "sqrt": np.sqrt(target.clip(lower=0)),
        "p1_3": target**(1/3),
        "p1_4": target**(1/4),
        "p1_5": target**(1/5),
        "log1p": np.log1p(target)
    }
    skewness = {key: val.skew() for key, val in transformations.items()}
    best_transformation = min(skewness, key=skewness.get)
    print(f'Best transformation is "{best_transformation}" with skewness {skewness[best_transformation]}')
    return transformations[best_transformation], best_transformation

In [87]:
def inverse_transform(predictions, transformations):
    if 'log1p' in transformations:
        return np.expm1(predictions)
    elif 'sqrt' in transformations:
        return np.power(predictions, 2)
    else:
        print('No transformation needed!')
        return predictions

In [88]:
def evaluate_model(model, X_train, y_train, X_test, y_test, transformations):
    model_name = model.__class__.__name__
    print(f"Scoring {model_name}")
    
    
    # Fit the model
    start = time.time()
    model.fit(X_train, y_train)
    endtime = time.time() - start
    
    # Predicting
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Inverse transform the predictions and true values only once
    y_train_original, y_test_original = inverse_transform(y_train, transformations), inverse_transform(y_test, transformations)
    y_pred_train_original, y_pred_test_original = inverse_transform(y_pred_train, transformations), inverse_transform(y_pred_test, transformations)
    
    # Calculate metrics
    metrics = {
        'Model': model_name,
        'RMSE Train Set': np.sqrt(mean_squared_error(y_train_original, y_pred_train_original)),
        'RMSE Test Set': np.sqrt(mean_squared_error(y_test_original, y_pred_test_original)),
        'MAE Train Set': mean_absolute_error(y_train_original, y_pred_train_original),
        'MAE Test Set': mean_absolute_error(y_test_original, y_pred_test_original),
        'R2 Train Set': r2_score(y_train_original, y_pred_train_original),
        'R2 Test Set': r2_score(y_test_original, y_pred_test_original),
        'Time(sec)': endtime
    }
    return metrics

In [89]:
def grid_search_for_best_model(grid_models, param_dict, scoring_dict, X_train, y_train):

    # Container for grid search results
    grid_results = []

    # Running grid search for each model
    for model_name, model in grid_models.items():
        print(f"Running GridSearchCV for {model_name}...")
        start = time.time()
        grid = GridSearchCV(model, param_grid=param_dict[model_name], cv=5, verbose=False,
                            scoring=scoring_dict, refit='MAE')
        
        grid.fit(X_train, y_train)
        fit_time = time.time() - start
        
        # Extracting best scores
        best_score_rmse = np.sqrt(-grid.cv_results_['mean_test_RMSE'].max())
        best_score_mae = -grid.cv_results_['mean_test_MAE'].max()
        best_score_r2 = grid.cv_results_['mean_test_R2'].max()
        
        # Appending results
        grid_results.append({
            'Model': model_name,
            'Best Parameters': grid.best_params_,
            'RMSE': best_score_rmse,
            'MAE': best_score_mae,
            'R2': best_score_r2,
            'Time(sec)': fit_time
        })

    # Converting results to DataFrame
    results_df = pd.DataFrame(grid_results)
    
    # Extracting the best model based on RMSE
    best_model_info = results_df.loc[results_df['RMSE'].idxmin()]
    best_model_name = best_model_info['Model']
    best_model = grid_models[best_model_name].set_params(**best_model_info['Best Parameters'])
    
    print(f"Best Model: {best_model_name} with parameters {best_model_info['Best Parameters']}")
    
    return best_model, results_df

In [90]:
def get_importances_expanded(preprocessor, best_model):
        """Extracts expanded feature importances from a trained model."""
        if not hasattr(best_model, 'feature_importances_'):
            print("The provided model does not have feature_importances_ attribute.")
            return None

        imp_list = []
        feature_names = preprocessor.get_feature_names_out()
        feature_importances = best_model.feature_importances_

        # Normalize the feature importances
        feature_importances_normalized = feature_importances / sum(feature_importances) * 100

        # Populate imp_list with feature names and their importances
        for name, importance in zip(feature_names, feature_importances_normalized):
            imp_list.append({'Importance Name': name, 'Importance(%)': round(importance, 2)})
        
        return pd.DataFrame(imp_list)

In [91]:
class ModelFeatureImportance:
    def __init__(self, feature_names, n_repeats=10, random_state=42):
        self.feature_names = feature_names
        self.n_repeats = n_repeats
        self.random_state = random_state
        self.feature_importances_data = []

    def aggregate_and_normalize_importances(self, importances):
        aggregated_importances = defaultdict(float)
        for feature, importance in zip(self.feature_names, importances):
            base_name = feature.split("__")[1] if "__" in feature else feature
            base_name = base_name.rsplit("_", 1)[0]
            aggregated_importances[base_name] += np.abs(importance)
        total_importance = sum(aggregated_importances.values())
        return {k: v / total_importance * 100 for k, v in aggregated_importances.items()}

    def process_model(self, model, X_test, y_test):
        model_name = type(model).__name__
        if hasattr(model, 'coef_'):
            coefs = model.coef_.flatten()
            importances = np.abs(coefs)
            importance_type = "Coefficient Importance"
        elif hasattr(model, 'feature_importances_'):
            importances = model.feature_importances_
            importance_type = "Feature Importance"
        else:
            result = permutation_importance(model, X_test, y_test, n_repeats=self.n_repeats, random_state=self.random_state)
            importances = np.abs(result.importances_mean)
            importance_type = "Permutation Importance"
        
        normalized_importances = self.aggregate_and_normalize_importances(importances)
        for feature, importance in normalized_importances.items():
            self.feature_importances_data.append((model_name, feature, importance, importance_type))

    def display_importances(self):
        for data in self.feature_importances_data:
            model_name, feature, importance, importance_type = data
            print(f"{model_name} - {importance_type} for {feature}: {importance:.2f}%")

In [92]:
def deploy_best_model(model, X_train, y_train, X_test, y_test):
    
    # Fit the model to the training data
    model.fit(X_train, y_train)
    
    # Make predictions on both the training and test sets
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Inverse the data
    y_train_original, y_test_original = inverse_transform(y_train, transformations), inverse_transform(y_test, transformations)
    y_pred_train, y_pred_test = inverse_transform(y_pred_train, transformations), inverse_transform(y_pred_test, transformations)
    
    # Combine the actual and predicted values for both training and test sets
    actual_combined = np.concatenate((y_train, y_test), axis=0)
    predicted_combined = np.concatenate((y_pred_train, y_pred_test), axis=0)
    
    # Create a DataFrame for the combined actual and predicted values
    final_output = pd.DataFrame({
        'Actual(USD)': actual_combined,
        'Predicted(USD)': predicted_combined
    })
    
    # Calculate errors and differences
    final_output['Dollar Difference(USD)'] = round(abs(final_output['Actual(USD)'] - final_output['Predicted(USD)']), 2)
    final_output['Error(%)'] = round(100 * final_output['Dollar Difference(USD)'] / final_output['Actual(USD)'], 2)
    
    # Calculate evaluation metrics for the combined dataset
    mape = final_output['Error(%)'].mean()
    rmse = np.sqrt(mean_squared_error(final_output['Actual(USD)'], final_output['Predicted(USD)']))
    mae = mean_absolute_error(final_output['Actual(USD)'], final_output['Predicted(USD)'])
    
    return mape, rmse, mae, final_output


---
## <u>Installations

In [93]:
# INSTALLATIONS
# !pip install pandas
# !pip install openpyxl
# !pip install numpy
# !pip install xgboost
# !pip install lightgbm
# !pip install category_encoders
# !pip install mlxtend
# !pip install shap

---
## <u>Imports of Libraries</u>
- imports separated for each step along the way

In [94]:
# CLEANING & BASIC IMPORTS
import pandas as pd
import numpy as np
import os
import time
from PyQt5.QtWidgets import QApplication, QFileDialog
from collections import defaultdict

# PREPROCESSING
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, OrdinalEncoder, PolynomialFeatures, LabelEncoder
from category_encoders import TargetEncoder, BinaryEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# MODELING
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.svm import SVR, LinearSVR

# EVALUATING
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import seaborn as sns
import shap

---
## <u>Reading File Into Project</u>
- taking into account for both 'xlsx' and 'csv' files
- display the full dataset rows and columns

In [95]:
df = grab_file()

CSV file selected. Reading file.


In [96]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

---
## <u>Data Cleaning</u>
- #### Only Keeping Necessary Columns for Analysis
- #### Removing Duplicates with Order ID the Same
    - False-Positive Transactions
- #### Renaming Columns for Similar Naming Conventions
- #### Filling missing values in 'promotion-ids' with "No Promotion"
- #### Drop missing values to disclude any potential biases
    - Since 'Amount' is crucial for analysis
    - There is only 1 type of currency wich is INR (Indian Rupee) * 0.012 for USD conversion
#### OPTIONAL:
- #### Extracting Valid Purchases from Orders
    - Ordered, Cancelled, Returned



In [97]:
df_cleaned = data_cleaning(df)

In [98]:
df_cleaned.head(2)

,Date,Category,Style,Size,SKU,Quantity,Amount,Promotions,B2B
48972,2022-03-31,Set,J,XS,J0127-SKD-XS,1,14.388,Free Shipping,0
49017,2022-03-31,kurta,JNE,XS,JNE3633-KR-XS,1,5.436,No Promotion,0


---
## <u>Feature Engineering

In [99]:
df_engineered = feature_engineering(df_cleaned)
df_engineered.head(2)

,Category,Style,Size,Quantity,Amount,Promotions,B2B,SKU_Color,Month,Day
0,Set,J,XS,1,14.388,Free Shipping,0,NOT LISTED,3,31
1,kurta,JNE,XS,1,5.436,No Promotion,0,NOT LISTED,3,31


---
## <u>Splitting Dataset</u>
- #### Train Test Split my Dataset
    - transfer target variable to split

In [100]:
X_train, X_test, y_train, y_test = split_dataset(df_engineered)
X_train.shape, X_test.shape

((84720, 9), (21180, 9))

---
## <u>Preprocessing (Encoding & Scaling)</u>

In [101]:
X_train, X_test, preprocessor = preprocess_features(X_train.copy(), X_test.copy())
X_train.shape, X_test.shape

((84720, 59), (21180, 59))

---
## <u>Addressing Skewness</u>
- #### Introducing functions to handle skewness of dataset
    
    - transforming target variable column and present a more evenly distributed data to analyze
    - will need to convert back to original form later

In [102]:
# Assuming apply_transformation is your function that applies the best transformation
y_train, _ = transform_target(y_train.copy())
y_test, _ = transform_target(y_test.copy())
y_train.skew(), y_test.skew()

(0.2456782846199202, 0.22925266829601365)

---
## <u>Developing a Baseline Model to Beat (DecisionTreeRegressor)

- #### Initializing baseline model(DecisionTreeRegressor) & models to beat the baseline

In [25]:
models = [
    DecisionTreeRegressor(random_state=42),
    XGBRegressor(),
    LGBMRegressor(force_col_wise=True, verbosity=-1),
    GradientBoostingRegressor(),
    RandomForestRegressor()
]

- #### Runnning models against baseline

In [26]:
model_scores = []
    
for model in models:
    
    # Evaluate the model
    metrics = evaluate_model(model, 
                             X_train, 
                             y_train, 
                             X_test, 
                             y_test, 
                             transformations)
    
    # Append the scoring with metrics dictionary
    model_scores.append(metrics)

Scoring DecisionTreeRegressor
Scoring XGBRegressor
Scoring LGBMRegressor
Scoring GradientBoostingRegressor
Scoring RandomForestRegressor


In [27]:
model_scores_df = pd.DataFrame(model_scores)
model_scores_df

,Model,RMSE Train Set,RMSE Test Set,MAE Train Set,MAE Test Set,R2 Train Set,R2 Test Set,Time(sec)
0,DecisionTreeRegressor,6.159424,6.178601,5.696946,5.695006,-327.762646,-321.639396,0.219351
1,XGBRegressor,6.059681,6.055635,5.657978,5.654694,-317.201179,-308.924894,0.262181
2,LGBMRegressor,6.038056,6.039500,5.649997,5.648124,-314.934167,-307.275445,0.214370
3,GradientBoostingRegressor,5.984069,5.980213,5.628486,5.623955,-309.309860,-301.252794,5.714258
4,RandomForestRegressor,6.132686,6.136255,5.686982,5.679709,-324.914555,-317.231999,13.275930


---
## <u>Grid Search Scoring

#### Setting up the Grid Search for the top 2 performing models (LGBMRegressor & XGBRegressor)

- The goal is to narrow down the best model to deploy

In [28]:
# Initialize Scoring Dictionary
scoring_dict = {
    'RMSE': make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), 
                        greater_is_better=False),
    'MAE': 'neg_mean_absolute_error',
    'R2': 'r2'
}

# Initialize Grid Models
grid_models = {
    'XGBoost': XGBRegressor(enable_categorical=True),
    'LightGBM': LGBMRegressor(force_col_wise=True, verbosity=-1),
}

# Setting parameters for grid search
param_dict = {

    'XGBoost': {'n_estimators': [460],
                'max_depth': [4, 5]},
#                 'learning_rate': [0.1],
#                 'reg_alpha': [0.01, 0.1],
#                 'reg_lambda': [2.4, 2.5, 2.6]},

    'LightGBM': {'num_leaves': [57, 58, 59, 60]},
                 'n_estimators': [740, 745, 750],
                 'max_depth': [6, 7, 8],
#                  'learning_rate': [0.1],
#                  'reg_alpha': [0.01, 0.1],
#                  'reg_lambda': [0.3, 0.4, 0.5]},
}

In [29]:
best_model, model_performance_df = grid_search_for_best_model(grid_models,
                                                              param_dict,
                                                              scoring_dict,
                                                              X_train, 
                                                              y_train)
print('The best model is:',best_model.__class__.__name__)

Running GridSearchCV for XGBoost...
Running GridSearchCV for LightGBM...
Best Model: LightGBM with parameters {'num_leaves': 58}
The best model is: LGBMRegressor


In [30]:
model_performance_df

,Model,Best Parameters,RMSE,MAE,R2,Time(sec)
0,XGBoost,"{'max_depth': 4, 'n_estimators': 460}",0.445243,0.149423,0.659385,8.031399
1,LightGBM,{'num_leaves': 58},0.444855,0.148846,0.660566,3.808651


---
## <u>Feature Importance</u>
#### Percentage Explainer
- Aggregating the transformed columns back to percentages

In [31]:
# Fit the best model to understand the feature importance
best_model.fit(X_train, y_train)

# Initialize the class with feature names
feature_importance_processor = ModelFeatureImportance(feature_names=preprocessor.get_feature_names_out())

# Process feature importances for each model
feature_importance_processor.process_model(best_model, X_test, y_test)

# Display the calculated feature importances
feature_importance_processor.display_importances()
importances_df = get_importances_expanded(preprocessor, best_model)
importances_df.sort_values(by='Importance(%)',ascending=False)

LGBMRegressor - Feature Importance for Category: 12.44%
LGBMRegressor - Feature Importance for Style: 11.82%
LGBMRegressor - Feature Importance for Size: 17.63%
LGBMRegressor - Feature Importance for Promotions: 13.35%
LGBMRegressor - Feature Importance for SKU_Color: 1.49%
LGBMRegressor - Feature Importance for Quantity: 2.75%
LGBMRegressor - Feature Importance for Month: 12.72%
LGBMRegressor - Feature Importance for Day: 27.11%
LGBMRegressor - Feature Importance for B2B: 0.68%


,Importance Name,Importance(%)
57,scale__Day,27.11
56,scale__Month,12.72
35,onehot__Promotions_Free-Financing,3.93
34,onehot__Promotions_Free Shipping,3.88
12,onehot__Style_J,3.86
36,onehot__Promotions_No Promotion,3.02
4,onehot__Category_Set,2.89
28,onehot__Size_S,2.84
30,onehot__Size_XS,2.79
55,scale__Quantity,2.75


---
## <u> Deploy Best Model on Test Data

In [34]:
mape, rmse, mae, final_output = deploy_best_model(best_model, 
                                                  X_train, 
                                                  y_train,
                                                  X_test, 
                                                  y_test)

# Display the metrics
print(f'Mean Absolute Percentage Error(MAPE): {mape}%')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
final_output.shape

Mean Absolute Percentage Error(MAPE): 259.1451525023607%
RMSE: 6.047657413123387
MAE: 5.653042326015045


(105900, 4)

In [ ]:
final_output.head()

---
## <u>Plotting the Model's Accuracy Over Time</u>